# Adding a transition matrix and Returns to E

In [68]:
import numpy as np
import pandas as pd


data_frame = pd.read_csv('data.respy.dat', delim_whitespace=True, header=0, na_values='.')
data_frame.set_index(['Identifier', 'Period'], drop=False, inplace=True)
num_periods = 40
num_agents_sim = 1000
data_frame.head()

num_agents_types = data_frame.groupby('Identifier').first()['Type'].value_counts().to_dict()
print num_agents_types


{0: 470, 1: 123, 2: 407}


In [70]:
choices = data_frame['Choice']

for t in range(num_periods):
    args = []
    for decision in [1, 2, 3, 4]:
        cond = data_frame['Type'] == 0
        args += [(choices.loc[slice(None), t] == decision).sum()]
    args = [x / float(num_agents_sim) for x in args]
        
choice_probs = dict()
num_types = 3
for type_ in range(num_types):
    choice_probs[type_] = []
    for t in range(num_periods):
        args = []
        for decision in [1, 2, 3, 4]:
            cond = (data_frame['Period'] == t) & (data_frame['Type'] == type_)
            args += [(data_frame.loc[cond, 'Choice'] == decision).sum()]
        args = [x / float(num_agents_types[type_]) for x in args]
        choice_probs[type_] += [args]

In [71]:
print choice_probs[0][0]
print choice_probs[1][0]
print choice_probs[2][0]

[0.38085106382978723, 0.10000000000000001, 0.51063829787234039, 0.0085106382978723406]
[0.88617886178861793, 0.0, 0.0, 0.11382113821138211]
[0.0049140049140049139, 0.081081081081081086, 0.91400491400491402, 0.0]
